In [87]:
import pandas as pd
import json
import sqlite3

In [88]:
orders_df=pd.read_csv("orders.csv")

In [68]:
orders_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [69]:
users_df=pd.read_json("users.json")

In [70]:
users_df.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [71]:
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()

In [72]:
restaurants_df=pd.read_sql("SELECT * FROM restaurants", conn)

In [73]:
restaurants_df.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [74]:
merged_df = pd.merge(orders_df, users_df, on="user_id", how="left")

In [75]:
final_df=pd.merge(merged_df, restaurants_df,on="restaurant_id",how="left")

In [76]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [80]:
final_df.to_csv("final_food_delivery_dataset.csv",index=False)

In [81]:
final_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [89]:
import sqlite3
import pandas as pd
conn =sqlite3.connect(":memory:")
df.to_sql("final_food_delivery_dataset", conn ,index=False, if_exists="replace")

NameError: name 'df' is not defined

In [84]:
query="""
SELECT city, 
SUM (order_amount) AS total_revenue
FROM final_food_delivery_dataset
WHERE membership_type= 'Gold'
GROUP BY city
ORDER BY total_revenue DESC
LIMIT 1;
"""

In [86]:
pd.read_sql(query,conn)

DatabaseError: Execution failed on sql '
SELECT city, 
SUM (order_amount) AS total_revenue
FROM final_food_delivery_dataset
WHERE membership_type= 'Gold'
GROUP BY city
ORDER BY total_revenue DESC
LIMIT 1;
': no such table: final_food_delivery_dataset

In [90]:
SELECT
    u.city,
    SUM(o.total_amount) AS total_revenue
FROM orders o
JOIN users u
    ON o.user_id = u.user_id
WHERE u.membership_type = 'Gold'
GROUP BY u.city
ORDER BY total_revenue DESC
LIMIT 1;



IndentationError: unexpected indent (3821465481.py, line 2)